In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium_stealth import stealth

In [2]:
import time
from numpy import random

In [6]:
%run gpt_handler.ipynb

['whisper-1', 'babbage', 'davinci', 'text-davinci-edit-001', 'babbage-code-search-code', 'text-similarity-babbage-001', 'code-davinci-edit-001', 'text-davinci-001', 'ada', 'babbage-code-search-text', 'babbage-similarity', 'code-search-babbage-text-001', 'text-curie-001', 'gpt-3.5-turbo-16k-0613', 'code-search-babbage-code-001', 'text-ada-001', 'text-similarity-ada-001', 'curie-instruct-beta', 'gpt-3.5-turbo-0301', 'ada-code-search-code', 'ada-similarity', 'code-search-ada-text-001', 'text-search-ada-query-001', 'davinci-search-document', 'ada-code-search-text', 'text-search-ada-doc-001', 'davinci-instruct-beta', 'text-similarity-curie-001', 'code-search-ada-code-001', 'ada-search-query', 'text-search-davinci-query-001', 'curie-search-query', 'davinci-search-query', 'babbage-search-document', 'ada-search-document', 'text-search-curie-query-001', 'gpt-3.5-turbo', 'text-search-babbage-doc-001', 'gpt-3.5-turbo-0613', 'curie-search-document', 'text-search-curie-doc-001', 'babbage-search-que

In [3]:
CHROMEDRIVER_PATH = "/executables/chromedriver.exe"
STACKOVERFLOW_LOGIN_URL = "https://stackoverflow.com/users/login"

In [4]:
def sleeprandom(minimum=0.5, maximum=1):
    st = random.rand() * (maximum - minimum) + minimum
    time.sleep(st)

In [5]:
class Webdriver_Handler:
    def __init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument('disable-infobars')
        options.add_experimental_option('useAutomationExtension', False)

        self.timeout = 10       

        service = Service(executable_path=CHROMEDRIVER_PATH)
        self.wd = webdriver.Chrome(service=service, options=options)
        
        self.wd.implicitly_wait(self.timeout)

        stealth(
            self.wd,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
        )
    
    def login(self, username, password):
        self.wd.get(STACKOVERFLOW_LOGIN_URL)
        sleeprandom()
        self.try_disabling_cookies()
        sleeprandom()
        self.wd.find_element(By.ID, "email").send_keys(username)
        sleeprandom()
        self.wd.find_element(By.ID, "password").send_keys(password)
        sleeprandom()
        self.wd.find_element(By.ID, "submit-button").click()
        sleeprandom()
    
    def try_disabling_cookies(self):
        try:
            self.wd.find_element(By.XPATH, """//button[contains(text(), "Necessary cookies only")]""").click()
            return True
        except:
            return False
    
    def is_logged_in(self):
        try: 
            username = self.wd.find_element(By.XPATH, """//a[@class="s-topbar--item s-user-card s-user-card__small m0 px12 js-gps-track"]/span""").text
            reputation = self.wd.find_element(By.XPATH,"""//li[contains(@title, "reputation")]""").text.split("\n")[0].replace(" ","")
            print(f"Logged in as {username} with {reputation} reputation")
            return True
        except:
            print("Not logged in")
            return False
    
    def upvote_questions(self, num_upvotes=3):
        self.wd.get("https://stackoverflow.com/questions/tagged/python")
        question_links = [a.get_attribute("href") for a in self.wd.find_elements(By.XPATH, """//div[@id="questions"]/div/div/h3/a""")]
        print(question_links)
        sleeprandom()

        upvotes_given = 0
        for question_link in question_links:
            self.wd.get(question_link)
            sleeprandom()
            vote_count = int(self.wd.find_element(By.XPATH, """//div[@id="question"]/div/div/div/div[contains(@class,"vote-count")]""").text.replace(" ",""))
            if vote_count < 0:
                print("Skipping question with negative votes")
                continue

            upvote_button = self.wd.find_element(By.XPATH, """//div[@id="question"]/div/div/div/button[contains(@class,"vote-up")]""")

            if upvote_button.get_attribute("aria-pressed") == "false":
                upvote_button.click()
                upvotes_given += 1
                print(f"Upvoted {question_link}")
                sleeprandom()
            else:
                print("Already upvoted")
            
            if upvotes_given >= num_upvotes:
                break
        
        print(f"Upvoted {upvotes_given} questions")
    
    def answer_questions(self, num_answers=3, answer_question_if_open=True):
        if answer_question_if_open and "stackoverflow.com/questions/" in self.wd.current_url and "tagged/python" not in self.wd.current_url:
            # question is already open
            self.answer_question()
        print(answer_question_if_open, "stackoverflow.com/questions/" in self.wd.current_url, "tagged/python" not in self.wd.current_url)
        self.wd.get("https://stackoverflow.com/questions/tagged/python?sort=Newest&filters=NoAnswers")
        question_links = [a.get_attribute("href") for a in self.wd.find_elements(By.XPATH, """//div[@id="questions"]/div/div/h3/a""")]
        print(question_links)
        sleeprandom()

        answers_given = 0
        for question_link in question_links:
            self.wd.get(question_link)
            sleeprandom()
            if self.answer_question():
                answers_given += 1
    
    def answer_question(self):
        num_answers_on_question = int(self.wd.find_element(By.XPATH, """//h2[@class="mb0"]""").get_attribute("data-answercount"))
        if num_answers_on_question > 0:
            print("Already answered")
            return False
        sleeprandom()
        question_title = self.wd.find_element(By.XPATH, """//a[@class="question-hyperlink"]""").text
        question_body = self.wd.find_element(By.XPATH, """//div[@id="question"]/div/div[2]/div[1]""").text
        print(question_title, "\n", question_body)
        answer = ask_for_stackoverflow_answer(question_title, question_body)
        print(answer)
        answer_area = self.wd.find_element(By.XPATH, """//textarea[@id="wmd-input"]""")
        for c in answer:
            answer_area.send_keys(c)
        sleeprandom()
        answer_button = self.wd.find_element(By.XPATH, """//button[@id="submit-button"]""")
        answer_button.click()
        sleeprandom()
        answer_is_posted = self.wd.find_element(By.XPATH, """//div[@id="answers"]//span[text()="Quantum"]""")
        if not answer_is_posted:
            print("Answer not posted")
            if input("Your answer is not posted, type anything to interrupt. Press enter to continue") == "":
                return False
            else:
                raise Exception("Answer not posted.")
        return True

